In [9]:
import nest_asyncio
nest_asyncio.apply()

In [10]:

import logfire
import os
import asyncio 
from IPython.display import display
from agents import Agent, Runner
from agents.extensions.visualization import draw_graph
from pydantic import BaseModel
from agents import Agent
from dataclasses import dataclass
import asyncio
from agents.extensions.visualization import draw_graph 
from openai import AsyncOpenAI 
from dotenv import load_dotenv  
load_dotenv() 
from agents import (
    Agent,
    Runner, 
    set_tracing_disabled,OpenAIChatCompletionsModel,enable_verbose_stdout_logging
)   
 
API_KEY=os.getenv("GEMINI_API_KEY")

if  not API_KEY  :
    raise ValueError("Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code.")

 
# logfire.configure(token=os.getenv('LOGFIR_TOKEN') ) 
# logfire.instrument_openai()
# enable_verbose_stdout_logging()
set_tracing_disabled(disabled=True)
API_KEY=os.getenv("GEMINI_API_KEY")

if  not API_KEY  :
    raise ValueError("Please set EXAMPLE_BASE_URL, EXAMPLE_API_KEY, EXAMPLE_MODEL_NAME via env var or code.")

 
client = AsyncOpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/",api_key=API_KEY,)

model = OpenAIChatCompletionsModel(model="gemini-2.0-flash",openai_client=client)




class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

@dataclass
class CalendarClass:
  story_theme:str

def calendar_def(wrapper,agent):
  return f"""Extract calendar events from text /n/n
          {wrapper.context.story_theme}
          """
  
async def main(theme): 
  
    ctx = CalendarClass(story_theme=theme)
     
    agent = Agent(
    name="Calendar extractor",
    instructions=calendar_def,
    model=model,
    output_type=CalendarEvent,
)
    result = await Runner.run(agent, input=theme,context=ctx) 
    content= result.final_output
    print(f"llm resp:{content}")
    return
 




if __name__ == "__main__":
    
    theme = "we have an event whose name is Agenticai on 19 oct 2025 and particepatent billgates elon musk, and andrew , and sir zia khan"

    asyncio.run(main(theme))


llm resp:name='Agenticai' date='19 oct 2025' participants=['billgates', 'elon musk', 'andrew', 'sir zia khan']
